In [2]:
friendship_info = {
    "benedict.s@hotmail.co.uk": ["jane.d@example.com", "emily.f@example.com"],  # Similar shopping times
    "alex.j@example.com": ["frank.g@example.com", "grace.h@example.com"],  # Similar shopping times
    "chris.t@example.net": ["harry.i@example.com"],  # Similar shopping times
    "jane.d@example.com": ["benedict.s@hotmail.co.uk", "emily.f@example.com"],  # Similar shopping times
    "dave.h@example.org": ["irene.j@example.com"],  # Similar shopping times
    "emily.f@example.com": ["benedict.s@hotmail.co.uk", "jane.d@example.com"],  # Similar shopping times
    "frank.g@example.com": ["alex.j@example.com"],  # Similar shopping times
    "grace.h@example.com": ["alex.j@example.com"],  # Similar shopping times
    "harry.i@example.com": ["chris.t@example.net"],  # Similar shopping times
    "irene.j@example.com": []}

# Extract unique email addresses (individuals) from the mock dataset
individuals = mock_df_corrected['customer_email'].unique()

# Initialize a DataFrame with zeros, using individuals as both rows and columns
friendship_matrix = pd.DataFrame(0, index=individuals, columns=individuals)

# Populate the matrix based on the friendship_info dictionary
for person, friends in friendship_info.items():
    for friend in friends:
        friendship_matrix.loc[person, friend] = 1
        friendship_matrix.loc[friend, person] = 1  # Friendship is mutual


In [3]:
# Assuming mock_df_corrected is our transaction dataset
features = mock_df_corrected.drop(['customer_email'], axis=1)  # Excluding email for feature encoding
features_encoded = pd.get_dummies(features)

In [4]:
# Assuming `features_encoded` is your encoded transaction data
# Let's first ensure the customer email is included for aggregation
mock_df_corrected['customer_email'] = mock_df_corrected['customer_email'].astype('category')
features_with_email = pd.concat([mock_df_corrected['customer_email'], features_encoded], axis=1)

# Aggregate features by customer email
individual_features = features_with_email.groupby('customer_email').mean()

# Ensure the index is consistent for later operations
individual_features = individual_features.reindex(friendship_matrix.index)

C:\Users\bened\AppData\Local\Temp\ipykernel_38248\4208845645.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  individual_features = features_with_email.groupby('customer_email').mean()


In [5]:
X_train_list = []
y_train_list = []

for i, email_i in enumerate(friendship_matrix.index):
    for j, email_j in enumerate(friendship_matrix.columns):
        if i < j:  # Avoid duplicate pairs and self-pairing
            # Combine features of both individuals in the pair
            features_pair = pd.concat([individual_features.loc[email_i], individual_features.loc[email_j]], axis=0).to_list()
            X_train_list.append(features_pair)
            # Corresponding friendship status
            y_train_list.append(friendship_matrix.loc[email_i, email_j])

# Convert lists to DataFrame and Series for X_train and y_train
X_train = pd.DataFrame(X_train_list)
y_train = pd.Series(y_train_list)


In [6]:
friendship_matrix

,benedict.s@hotmail.co.uk,alex.j@example.com,chris.t@example.net,jane.d@example.com,dave.h@example.org,emily.f@example.com,frank.g@example.com,grace.h@example.com,harry.i@example.com,irene.j@example.com
benedict.s@hotmail.co.uk,0,0,0,1,0,1,0,0,0,0
alex.j@example.com,0,0,0,0,0,0,1,1,0,0
chris.t@example.net,0,0,0,0,0,0,0,0,1,0
jane.d@example.com,1,0,0,0,0,1,0,0,0,0
dave.h@example.org,0,0,0,0,0,0,0,0,0,1
emily.f@example.com,1,0,0,1,0,0,0,0,0,0
frank.g@example.com,0,1,0,0,0,0,0,0,0,0
grace.h@example.com,0,1,0,0,0,0,0,0,0,0
harry.i@example.com,0,0,1,0,0,0,0,0,0,0
irene.j@example.com,0,0,0,0,1,0,0,0,0,0


In [7]:
from tpot import TPOTClassifier
import sklearn.model_selection

# Assuming your X_train and y_train are correctly prepared
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_train, y_train, 
                                                                            test_size=0.25, random_state=42)

# Initialize TPOT classifier
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)

# Fit the TPOT model to your data
tpot.fit(X_train, y_train)

# Evaluate the model on the test set
print(tpot.score(X_test, y_test))

# Optionally, export the pipeline to a Python file
# tpot.export('tpot_best_pipeline.py')


c:\Users\bened\AppData\Local\Programs\Python\Python312\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


                                                                              
Generation 1 - Current best internal CV score: 0.8476190476190476
                                                                              
Generation 2 - Current best internal CV score: 0.8476190476190476
                                                                              
Generation 3 - Current best internal CV score: 0.8476190476190476
                                                                              
Generation 4 - Current best internal CV score: 0.8476190476190476
                                                                              
Generation 5 - Current best internal CV score: 0.8476190476190476
                                                                              
Best pipeline: XGBClassifier(input_matrix, learning_rate=0.001, max_depth=9, min_child_weight=7, n_estimators=100, n_jobs=1, subsample=0.45, verbosity=0)
0.8333333333333334


In [23]:
print(tpot.score(X_test, y_test))


0.6666666666666666


In [ ]:
###Bits to add
    # How frequently a customer buys - will have to calculate from basket data then put into customer row
    # Weight it all to include last X sets of transactions 